In [11]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from sentence_transformers import SentenceTransformer
from model.decoderMLP import decoderMLP, decoderAttention, movieTransformer
from tqdm import tqdm
import openai
import pickle 
import argparse
from typing import List
import logging
import numpy as np
import torch
import json
import time
import torch.optim as optim
import torch
from pprint import pprint as pp
from scipy.sparse import csr_matrix
import os 
import pandas as pd
from collections import defaultdict
from helper.sampler import NegSampler, negsamp_vectorized_bsearch_preverif
from argparse import ArgumentParser
from model.MF import MatrixFactorization,MatrixFactorizationLLM
from trainer.training_utils import *
from helper.eval_metrics import *
from helper.dataloader import *

In [4]:
lr= 0.00001
epochs = 400
num_heads = 6
cosine = False
num_layers = 3
output_emb = 64
embedding_dim = 768
saved_path = f'../saved_model_cluster/ml-100k/attn_best_model_{lr}_{epochs}_{num_heads}_{cosine}_{num_layers}.pth'

model_path = saved_path + '_best_model.pth'
embedder_path = saved_path + '_embedder.pth'
item_embeddings_path = saved_path + '_item_embeddings.pth'
user_embeddings_path = saved_path + '_user_embeddings.pth'
model_rankings_path = saved_path + '_rankings_matrix.npy'
id_genre_map = map_id_to_genre('../data/ml-100k/movies.dat')

# 1. Data Loading & Preprocessing
train_data = load_dataset("../data_preprocessed/ml-100k/data_split/train_set_leave_one.json")
valid_data = load_dataset("../data_preprocessed/ml-100k/data_split/valid_set_leave_one.json")
test_data = load_dataset("../data_preprocessed/ml-100k/data_split/test_set_leave_one.json")
movie_title_to_id = map_title_to_id("../data/ml-100k/movies.dat")

train_data = convert_titles_to_ids(train_data, movie_title_to_id)
valid_data = convert_titles_to_ids(valid_data, movie_title_to_id)
test_data = convert_titles_to_ids(test_data, movie_title_to_id)

train_matrix, actual_list_val, actual_list_test = create_train_matrix_and_actual_lists(train_data, valid_data,
                                                                                        test_data, movie_title_to_id)
train_matrix = csr_matrix(train_matrix)  # Convert train_matrix to a CSR matrix


# 2. Model Creation
num_users, num_items = train_matrix.shape

In [5]:
args = parse_args(True)
user_embedder = decoderAttention(embedding_dim,num_heads,num_layers,output_emb, 0  )

model = MatrixFactorizationLLM(num_users, user_embedder,num_items, args).to(args.device)


/Users/emilianopenaloza/Git/LLM4Rec/notebooks/../model/MF.py:11: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [35]:
rankings_true = np.load(model_rankings_path)

In [7]:
model.load_state_dict(torch.load(model_path,map_location=torch.device('mps')))
user_embedder.load_state_dict(torch.load(user_embeddings_path,map_location=torch.device('mps')))
model.user_embeddings = user_embedder

model.eval()



MatrixFactorizationLLM(
  (user_embeddings): decoderAttention(
    (attnModule): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
    )
    (mlp_shrinker): decoderMLP(
      (mlp): Sequential(
        (0): Linear(in_features=768, out_features=448, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.25, inplace=False)
        (3): Linear(in_features=448, out_features=128, bias=True)
        (4): ReLU()
        (5): Dropout(p=0.25, inplace=False)
        (6): Identity()
        (7): Linear(in_features=128, out_features=64, bias=True)
        (8): Dropout(p=0.25, inplace=False)
      )
    )
  )
  (item_embeddings): Embedding(1682, 64)
)

In [67]:
summaries = {
    "Comedy": "Summary: This list of comedy movies includes adventure, drama, science fiction, and romance. Set in various locations, these films portray the trials, tribulations, and humorous situations of diverse characters in different settings.",
    "Romance": "Summary: A collection of lighthearted romantic comedies and dramas set in various locations and time periods, showcasing diverse relationships and the challenges they face.",
    "Drama": "Summary: A collection of drama films from the 1990s, exploring various themes such as historical mysteries, trials and tribulations of life in poverty-stricken neighborhoods, LGBT-related stories, romantic comedies, biographical tales, and epic journeys.",
    "Action": "Summary: Action movies with a dark and gritty tone, featuring intense crime stories and strong performances from the cast."
    }
  
  
def get_preds(summaries,top_k = 20 ): 
    args.embedding_module = 't5'
    embs = get_genrewise_embeddings(summaries,args, model= True)
    genre_list = get_genres()
    embs_tens = model.user_embeddings.prepare_input(embs,genre_list).to(args.device)

    scores = model.predict(embs_tens.unsqueeze(0)).cpu().detach().numpy()
    ranked_items = np.argsort(scores)[::-1]
    ranked_items = list(reversed(ranked_items[0] +1))

    recall_val = recall_at_k_one(actual_list_val[3], ranked_items, 20)
    print(f"checking recall: {recall_val=}")
    reversed_movie_title_to_id = {v: k for k, v in movie_title_to_id.items()}
    movie_titles_ranked = [f'{index} : {reversed_movie_title_to_id[i+1]} {id_genre_map[i+1]}' for index,i in enumerate(ranked_items[:20])]
    pp(movie_titles_ranked)
    return ranked_items[:20]

In [67]:
get_preds(summaries)

hi
ranked_items=[560, 786, 58, 1596, 53, 696, 790, 12, 825, 730, 1, 1421, 1041, 1551, 274, 963, 782, 547, 783, 724, 789, 785, 1518, 374, 787, 1239, 958, 1042, 764, 1148, 1208, 276, 1110, 23, 1157, 1073, 1407, 317, 1058, 52, 1209, 81, 1068, 1043, 1098, 54, 1167, 735, 549, 1195, 1409, 1220, 697, 363, 1153, 1408, 1354, 406, 398, 1297, 28, 458, 1268, 1426, 86, 1305, 557, 1519, 47, 591, 376, 1186, 562, 49, 1179, 365, 57, 78, 739, 478, 1627, 129, 1227, 1072, 727, 843, 1066, 1230, 956, 1112, 1425, 62, 1595, 464, 1443, 788, 1004, 1423, 1101, 419, 1312, 1621, 931, 416, 1063, 1474, 1480, 386, 110, 833, 364, 382, 1210, 791, 147, 1637, 5, 64, 545, 88, 548, 378, 997, 1185, 614, 67, 842, 135, 380, 2, 699, 66, 1337, 606, 1471, 404, 1040, 932, 173, 381, 771, 1115, 356, 1328, 1438, 1351, 1355, 397, 51, 540, 609, 700, 1287, 1052, 598, 469, 793, 979, 1503, 781, 120, 489, 1269, 1458, 717, 1495, 765, 588, 607, 775, 1017, 1577, 1396, 1194, 566, 43, 70, 584, 131, 383, 718, 564, 692, 495, 166, 622, 602, 1470,

In [87]:
summaries = {
    "Children": "Summary: animated toy adventures.",  
    }
  
get_preds(summaries)




hi
ranked_items=[58, 560, 1596, 786, 1421, 730, 53, 790, 782, 785, 1551, 789, 958, 787, 696, 562, 1195, 52, 825, 1073, 1407, 1043, 1058, 1297, 54, 1148, 1110, 1208, 557, 963, 1518, 1268, 1157, 1209, 1153, 788, 783, 57, 67, 1167, 1220, 1230, 478, 1041, 1408, 86, 458, 1354, 1239, 12, 1305, 382, 1426, 1, 1068, 1519, 1409, 1269, 724, 1101, 931, 464, 62, 833, 1112, 274, 979, 317, 88, 997, 1063, 1637, 110, 1505, 66, 28, 1423, 1042, 735, 1424, 602, 276, 81, 380, 364, 699, 23, 547, 131, 1312, 374, 356, 1210, 614, 1098, 793, 1337, 213, 363, 932, 383, 2, 566, 1186, 51, 598, 982, 1072, 1438, 1474, 1020, 1115, 1627, 697, 1503, 1443, 791, 381, 764, 378, 146, 79, 1179, 778, 549, 465, 49, 479, 781, 480, 1675, 1004, 731, 604, 418, 792, 406, 727, 202, 564, 1445, 365, 129, 836, 1621, 1552, 821, 548, 173, 1303, 237, 1147, 376, 744, 956, 415, 591, 715, 469, 64, 1328, 540, 832, 212, 625, 843, 398, 78, 1227, 652, 1066, 545, 47, 273, 693, 135, 584, 1355, 325, 164, 21, 65, 1314, 524, 156, 1480, 416, 609, 456,

In [85]:
summaries = {
    "Children": "Summary: animated toy adventures.",  
    "Comedy": "Summary: A collection of comedy films spanning different eras and styles, featuring romantic entanglements, humorous misadventures, and comedic performances from an ensemble of actors.",
    }
  
get_preds(summaries)


hi
ranked_items=[58, 560, 1596, 790, 1421, 786, 53, 730, 782, 1551, 785, 1148, 787, 789, 958, 54, 1073, 562, 52, 1043, 825, 1407, 557, 1110, 1058, 1195, 1297, 963, 1153, 1208, 1157, 1518, 57, 1268, 696, 788, 1167, 67, 1230, 1354, 478, 1220, 1426, 783, 1408, 1305, 382, 931, 833, 1269, 979, 1209, 1112, 1041, 1063, 1637, 458, 464, 1068, 66, 1409, 997, 380, 1101, 110, 62, 1423, 1239, 1424, 88, 1312, 602, 793, 699, 86, 1519, 932, 28, 317, 1505, 614, 1, 1042, 356, 1072, 383, 364, 131, 81, 274, 735, 566, 724, 12, 146, 1337, 1438, 982, 51, 276, 1020, 23, 1210, 480, 213, 381, 547, 1098, 792, 1627, 697, 1115, 1503, 598, 564, 1474, 479, 1445, 604, 731, 202, 2, 378, 791, 1186, 465, 1675, 79, 778, 821, 781, 363, 418, 173, 832, 49, 764, 237, 1179, 1004, 1621, 548, 652, 836, 1147, 625, 1443, 374, 212, 727, 406, 524, 744, 129, 1552, 415, 843, 376, 1303, 545, 325, 1480, 365, 456, 164, 469, 591, 693, 1227, 1314, 540, 1328, 135, 794, 980, 549, 956, 273, 78, 519, 1495, 622, 603, 416, 1066, 65, 1180, 481, 

In [5]:
args = get_args()
editor = RobertaEditor(args).to(device)

scorer = SteepHC(args,editor)

Editor built


In [1]:
data  = ''' Drama: Summary: A collection of drama films that explore themes of love, personal growth, and societal challenges. Each film tells a unique story with memorable characters and emotional depth.
    Romance: Summary: A collection of romance films with diverse themes, including a Cuban refugee pretending to be a family, an erotic romantic thriller, a fantasy comedy about reliving the same day, a mistaken identity romantic comedy, a classic screwball comedy, a Shakespeare
    Children: Summary: These children's movies are filled with fantasy, adventure, and heartwarming stories. From animated tales to magical adventures, these films are perfect for young viewers.
    Comedy: Summary: A collection of comedy films spanning different eras and styles, featuring romantic entanglements, humorous misadventures, and comedic performances from an ensemble of actors.
    Action: Summary: Action-packed films with a mix of genres including Western, medical disaster, buddy cop, science fiction, and horror anthology. The movies feature intense action sequences and diverse plots involving epic space battles, covert operations, and post-apocalyptic worlds.
    Thriller: Summary: A collection of thrilling films with elements of psychological drama, legal suspense, science fiction, and horror. Featuring talented actors and directors, these movies keep audiences on the edge of their seats with intense storylines and unexpected twists.
    Crime: Summary: A collection of crime films with various themes, including violence, drama, comedy, and thrillers, featuring notable directors and actors.
    Adventure: Summary: Adventure movies filled with thrilling journeys, time travel, and epic battles. From exploring other planets through a mystical portal to going back in time or diving into a fantastical Arthurian world, these films take us on incredible escapades.
    Sci-Fi: Summary: A collection of science fiction films that transport audiences to different worlds and challenge their perceptions of reality. These movies explore themes of adventure, disaster, and the power of technology in intriguing and unexpected ways.
    Fantasy: Summary: Adventure comedies with a fantasy twist that provide an enjoyable escape from reality.
'''

In [2]:
text = """Drama: Summary: A collection of drama films that explore themes of love, personal growth, and societal challenges. Each film tells a unique story with memorable characters and emotional depth.
    Romance: Summary: A collection of romance films with diverse themes, including a Cuban refugee pretending to be a family, an erotic romantic thriller, a fantasy comedy about reliving the same day, a mistaken identity romantic comedy, a classic screwball comedy, a Shakespeare
    Children: Summary: These children's movies are filled with fantasy, adventure, and heartwarming stories. From animated tales to magical adventures, these films are perfect for young viewers.
    Comedy: Summary: A collection of comedy films spanning different eras and styles, featuring romantic entanglements, humorous misadventures, and comedic performances from an ensemble of actors.
    Action: Summary: Action-packed films with a mix of genres including Western, medical disaster, buddy cop, science fiction, and horror anthology. The movies feature intense action sequences and diverse plots involving epic space battles, covert operations, and post-apocalyptic worlds.
    Thriller: Summary: A collection of thrilling films with elements of psychological drama, legal suspense, science fiction, and horror. Featuring talented actors and directors, these movies keep audiences on the edge of their seats with intense storylines and unexpected twists.
    Crime: Summary: A collection of crime films with various themes, including violence, drama, comedy, and thrillers, featuring notable directors and actors.
    Adventure: Summary: Adventure movies filled with thrilling journeys, time travel, and epic battles. From exploring other planets through a mystical portal to going back in time or diving into a fantastical Arthurian world, these films take us on incredible escapades.
    Sci-Fi: Summary: A collection of science fiction films that transport audiences to different worlds and challenge their perceptions of reality. These movies explore themes of adventure, disaster, and the power of technology in intriguing and unexpected ways.
    Fantasy: Summary: Adventure comedies with a fantasy twist that provide an enjoyable escape from reality."""

# Split the text into words
words = text.split()

# Initialize a list to store the indices of words with ':'
indices_of_words_with_colon = []

# Iterate through the words to find the indices of words with ':'
for index, word in enumerate(words):
    if ':' in word:
        # Add the index to the list
        indices_of_words_with_colon.append(index)

# Print the list of indices of words with ':'
print("Indices of words with ':' in the list:", indices_of_words_with_colon)

Indices of words with ':' in the list: [0, 1, 29, 30, 71, 72, 97, 98, 122, 123, 160, 161, 199, 200, 220, 221, 260, 261, 297, 298]


In [3]:
def make_string_dict(string):
        # Split the text into genre entries
    genre_entries = text.strip().split('\n')

    # Initialize a dictionary to store the JSON data
    genre_data = {}

    # Iterate through the genre entries and populate the dictionary
    for entry in genre_entries:
        lines = entry.strip().split('\n')
        genre_name, genre_description = lines[0], lines[1:]
        genre_data[genre_name] = " ".join(genre_description)
    return genre_data

In [4]:

def style_ranker(text_new,original_text,movie_id =1073 ,original_index =16 ):
    preds_new = get_preds(make_string_dict(text_new))
    preds_old = get_preds(make_string_dict(original_text))
    if 1073 in preds_new: 
        print(f"{preds_new.index(1073) > preds_old.index(1073)=}")
        print(f"{preds_old.index(1073)=}")
        print(f"{preds_new.index(1073)=}")
        if preds_new.index(1073) > preds_old.index(1073):
            return 1
    else:
        return 0
    
    
   

In [5]:
data =[data]

In [6]:
%load_ext autoreload
%autoreload 2
import math
import os
import sys 
import logging
sys.path.append('../')
import torch.multiprocessing as mp
import warnings
import datetime
from dateutil import tz
import torch
# from transformers import RobertaModel, RobertaTokenizer
from StyleTransfer.scorer import *
from StyleTransfer.editor import RobertaEditor


os.environ["TOKENIZERS_PARALLELISM"] = "false"
tzone = tz.gettz('')
warnings.filterwarnings('ignore')
device = torch.device("mps")

args = get_args()

editor = RobertaEditor(args).to(device)


Editor built


In [7]:
sahc = SteepHC(args, editor).to(device)
of_dir = 'results/' + args.output_dir
if not os.path.exists(of_dir):
    os.makedirs(of_dir)

if args.direction == '0-1': postfix = '0'
else: postfix = '1'

filename='../StyleTransfer/data/{}/test.{}'.format(args.dst,postfix)
# with open(filename, 'r', encoding='utf8') as f:
#     data = f.readlines()[:]

bsz = args.bsz
max_len=len(data[0])
dst=args.dst
num_batches = math.ceil(len(data) / float(bsz))
timestamp = datetime.datetime.now().astimezone(tzone).strftime('%Y-%m-%d_%H:%M:%S')

output_file =f'{timestamp}_{dst}_seed={str(args.seed)}_{str(args.style_weight)}_{args.direction}.txt'

log_txt_path=os.path.join(of_dir, output_file.split('.txt')[0] + '.log')
print(log_txt_path)


results/output/2023-11-10_10:48:51_yelp_seed=42_8_0-1.log


In [8]:
max_len

2239

In [14]:
import random
import ray
from pprint import pprint as pp
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(format='',
                    filename=log_txt_path,
                    filemode='w',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

word_pairs ={"ca n't": "can not", "wo n't": "will not"}
logging.info(args)

def print_es():
    print("Early Stopping!")
    logging.info("Early Stopping!")


with open(of_dir + output_file, 'w', encoding='utf8') , torch.no_grad():
    for i in range(num_batches):
        batch_data = data[bsz * i:bsz * (i + 1)]

        # print(f"{batch_data=}")

        #preprocessing
        ref_oris = []
        for d in batch_data:
            for k, v in word_pairs.items():
                    d=d.strip().lower().replace(k, v)
            ref_oris.append(d)

        ref_olds=ref_oris.copy()
        state_vec, _ = editor.state_vec(ref_olds)


        break_flag = False
        max_score=0
        step_max_score_list=[0]
        seq_len=[len(line.split()) for line in ref_olds]
        max_seq_len=max(seq_len)

        for step in range(args.max_steps):
            #get the whole candidate list
            sampled_indices = random.sample(range(max_len), 1)
            input_tuples = [[ref_olds,[ops]*bsz,[positions]*bsz,bsz,max_len]
                                                    for positions in sampled_indices if positions not in indices_of_words_with_colon for ops in [0,1,2]]


            ref_news = [editor.edit(*inp)for inp in input_tuples[:1]]

                                                    

            for idx in range(len(ref_news)):

                ref_new_batch_data=ref_news[idx]

                # Calculating the acceptance probability

                index, ref_old_score, ref_new_score, new_style_labels,_ \
                    = sahc.acceptance_prob(ref_new_batch_data, ref_olds, ref_oris, state_vec)

                ref_hat = ref_new_batch_data[index]
                new_style_label=new_style_labels[index]
                
                # Updating the maximum score and selected sentence
                if ref_new_score>max_score and ref_new_score>ref_old_score:
                    max_score=ref_new_score
                    select_sent = ref_hat

                # the style is changed!
                if args.early_stop == True:
                    if (args.direction == '0-1' and new_style_label == 1) or \
                            (args.direction == '1-0' and new_style_label == 0) :
                        select_sent = ref_hat
                        print_es()
                        break_flag = True
                        break
            
            # Checking if the current score is larger than previous max score
            if max_score>step_max_score_list[step]: 
                print("hill climbing!")
                logging.info("hill climbing!")
                ref_olds = [select_sent]
                step_max_score_list.append(max_score.item())
            else:
                print("don't climb, stop!")
                logging.info("don't climb, stop!")
                break_flag=True

            if break_flag:
                break

        if break_flag:
            select_sent = select_sent

        logging.info('climb {} steps, the selected sentence is: {}'.format(step+1,select_sent))
        print('climb {} steps, the selected sentence is: {}'.format(step+1,select_sent))


        logging.info('\n')
     




RuntimeError: MPS backend out of memory (MPS allocated: 35.23 GB, other allocations: 1.07 GB, max allowed: 36.27 GB). Tried to allocate 84.55 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).